The purpose of this Notebook to add the latitude and the longitude coordinates of each neighborhood to the dataframe created in the Segmenting and Clustering Neighborhoods in Toronto_part1.ipynb notebook so the Foursquare location data can be utilized

#### Importing required libraries


In [1]:
import pandas as pd
import requests   
import lxml
from bs4 import BeautifulSoup       


#### Scraping the Wikipedia page and conerting its data into a pandas dataframe with three columns: Postalcode, Borough, Neighborhood

In [3]:
raw_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(raw_data,'lxml')
table=soup.find('table', attrs={'class':'wikitable sortable'})
df = df = pd.DataFrame(columns = ['Postalcode','Borough','Neighborhood'])
rows = table.find_all('tr')
for row in rows:
    data=[]
    for td in row.find_all('td'):
        data.append(td.text.strip())
    if len(data)==3:
        df.loc[len(df)] = data


#### Only the cells that have an assigned borough are processed and therefore the cells with a borough that is Not assigned are ignored

In [4]:
df = df[df.Borough != "Not assigned"]

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [5]:
df['Neighborhood'] = df.Neighborhood.apply(lambda x: x if not pd.isnull(x) else df.Borough)

#### Reading the csv file that includes the geographical coordinates of each postal code into a new data frame

In [7]:
coordinates_df=pd.read_csv('http://cocl.us/Geospatial_data')
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We rename the "Postal Code" column to "Postalcode" so we can merge it with the df dataframe

In [9]:
coordinates_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
new_df = pd.merge(df, coordinates_df, on='Postalcode')
new_df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
